# Chatbot de voz con Deep Learning

In [ ]:
!git clone https://github.com/AcecomFCUNI/Chatbot-Acecom.git
%cd Chatbot-Acecom/src

## 1 - El problema a resolver

La idea es crear un chatbot que interprete voz humana y genere la conversación en formato texto, usando las <ins>mejores arquitecturas de Deep Learning disponibles</ins>:

![](./../assets/idea_general_chatbot.png)

## 2 - Elementos del chatbot

Usaremos *wav2vec2* para la conversión voz a texto, y *BlenderBot* para generar la conversación:

![](./../assets/chatbot_detallado.png)

Tanto *wav2vec2* como *BlenderBot* se basan en las [Redes Transformer](https://youtu.be/Wp8NocXW_C4):

![](./../assets/red-transformer.png)

## 3 - Conversión voz a texto con *wav2vec2*

[*wav2vec2*](https://arxiv.org/pdf/2006.11477.pdf) fue desarrollado por Facebook en 2020:

![](./../assets/wav2vec2.png)



In [1]:
!pip install transformers #wav2vec2 y blenderbot
!pip install git+git://github.com/ricardodeazambuja/colab_utils.git #mic
!pip install librosa # pre-procesamiento audio

     |████████████████████████████████| 2.5MB 7.2MB/s 
     |████████████████████████████████| 3.3MB 53.4MB/s 
     |████████████████████████████████| 901kB 31.5MB/s 
  Cloning git://github.com/ricardodeazambuja/colab_utils.git to /tmp/pip-req-build-c3ig045t
  Running command git clone -q git://github.com/ricardodeazambuja/colab_utils.git /tmp/pip-req-build-c3ig045t
  Created wheel for colab-utils: filename=colab_utils-0.2-cp37-none-any.whl size=21465 sha256=d757cbb2edb71ee63960e4944aa49d197c58ba2af94952a22b1a032e4e5d6763
  Stored in directory: /tmp/pip-ephem-wheel-cache-0mpj5fhu/wheels/21/75/32/38aeb76b2424385f43eae9fd28c98e084308f4f6d9cb0a4f97
Successfully built colab-utils


In [ ]:
# Importar librerías
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
from colab_utils import getAudio
import librosa
import numpy as np

w2v2 = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
w2v2_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Capturar audio del mic (a 48 KHz)
audio, sr = getAudio()

In [ ]:
# Cambiar tasa de muestreo a 16 KHz (requerido por wav2vec2)
audio_float = audio.astype(np.float32)
audio_16k = librosa.resample(audio_float, sr, 16000)
print(f'Tamaño audio original: {audio_16k.shape}')

# Voz a texto
entrada = w2v2_processor(audio_16k, sampling_rate=16000, return_tensors="pt").input_values
print(f'Tamaño entrada a wav2vec2: {entrada.shape}')
probabilidades = w2v2(entrada).logits
print(f'Tamaño arreglo probabilidades (salida de wav2vec2): {probabilidades.shape}')
predicciones = torch.argmax(probabilidades, dim=-1)
print(f'Tamaño arreglo predicciones: {predicciones.shape}')
transcripcion = w2v2_processor.decode(predicciones[0])
print(transcripcion)

Tamaño audio original: (65280,)
Tamaño entrada a wav2vec2: torch.Size([1, 65280])
Tamaño arreglo probabilidades (salida de wav2vec2): torch.Size([1, 203, 32])
Tamaño arreglo predicciones: torch.Size([1, 203])
THIS IS JUST A TEST


## 4 - *BlenderBot*



[*BlenderBot*](https://ai.facebook.com/blog/state-of-the-art-open-source-chatbot/) también fue desarrollado por FaceBook en 2020, con el fin de permitir una interacción más humana y natural:

![](./../assets/blenderbot.png)

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("facebook/blenderbot-400M-distill")
blender = AutoModelForSeq2SeqLM.from_pretrained("facebook/blenderbot-400M-distill")

In [6]:
blender.generate?

In [ ]:
# Prueba inicial
entradaBlender = tokenizer([transcripcion], return_tensors='pt')
print(f'Frase de entrada: {transcripcion}')
print(f'Entrada a BlenderBot: {entradaBlender}')
ids_respuesta = blender.generate(**entradaBlender)
print(f'Salida BlenderBot: {ids_respuesta}')
respuesta = tokenizer.batch_decode(ids_respuesta)
print(f'Salida después del Tokenizer: {respuesta}')

Frase de entrada: THIS IS JUST A TEST
Entrada a BlenderBot: {'input_ids': tensor([[5760, 2566,  587, 6583,  349,  327, 2291,   59,    2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


Salida BlenderBot: tensor([[   1,    1,  281,  513,   19,  281,  632,  394, 3424,   21,  228,  281,
          360,  635, 2555,  335,  381,  335,  265,  816,  552,   21,    2]])
Salida después del Tokenizer: ['<s><s> I know, I am so excited.  I have been waiting for this for a long time.</s>']


In [ ]:
# Eliminar tokens de inicio y finalización de frase
respuesta = respuesta[0].replace('<s>','').replace('</s>','')
print(f'Salida en el formato correcto: {respuesta}')

Salida en el formato correcto:  I know, I am so excited.  I have been waiting for this for a long time.


In [ ]:
# Crear un corto chat de prueba
NFRASES = 5
nfrase = 1
while nfrase <= NFRASES:
  frase = input('-> MIGUEL: ')
  entradaBlender = tokenizer([frase], return_tensors='pt')
  ids_respuesta = blender.generate(**entradaBlender)
  respuesta = tokenizer.batch_decode(ids_respuesta)
  respuesta = respuesta[0].replace('<s>','').replace('</s>','')
  print(f'-> BLENDERBOT: {respuesta}')

  nfrase += 1

-> MIGUEL: Hi, how are you? Are you there?
-> BLENDERBOT:  I am here, and I am doing well. How are you this fine evening? 
-> MIGUEL: I'm just having dinner. What are you doing?
-> BLENDERBOT:  I'm eating dinner as well. I'm making spaghetti and meatballs.
-> MIGUEL: Mmmm.... I love spaghetti. Do you know any other recipes with spaghetti?
-> BLENDERBOT:  I do not, but I do know that it is one of the most popular foods in the world.
-> MIGUEL: Do you love spicy food?
-> BLENDERBOT:  Yes, I do.  I love the spicy flavor of the food.  Do you like spicy foods?
-> MIGUEL: No, I hate it! I prefer more salty food!
-> BLENDERBOT:  I do too, but it was so good.  I was so disappointed in myself.


## 5 - *wav2dec2* + *BlenderBot* y prueba del chatbot

Ahora introduciremos la captura de audio -> wav2dec2 -> BlenderBot en un loop:

In [ ]:
NFRASES = 5
nfrase = 1

while nfrase <= NFRASES:
  input()     # Esperar a pulsar tecla para iniciar grabación
  
  # Capturar audio y llevarlo a 16 KHz
  audio, sr = getAudio()
  audio_float = audio.astype(np.float32)
  audio_16k = librosa.resample(audio_float, sr, 16000)

  # Voz a texto
  entrada = w2v2_processor(audio_16k, sampling_rate=16000, return_tensors="pt").input_values
  probabilidades = w2v2(entrada).logits
  predicciones = torch.argmax(probabilidades, dim=-1)
  frase = w2v2_processor.decode(predicciones[0])
  
  # Imprimir transcripción
  print(f'-> MIGUEL: {frase}')

  # BlenderBot
  entradaBlender = tokenizer([frase], return_tensors='pt')
  ids_respuesta = blender.generate(**entradaBlender)
  respuesta = tokenizer.batch_decode(ids_respuesta)
  respuesta = respuesta[0].replace('<s>','').replace('</s>','')
  print(f'-> BLENDERBOT: {respuesta}')

  nfrase += 1

-> MIGUEL: HI ARE YOU DOING ARE YOU THERE
-> BLENDERBOT:  No, I am not.  I am at work.  What are you up to?



-> MIGUEL: IAM JUST HERE AT HAME SITTING SITTED ON THE COUCH AND WATCHING PSALM AND NEPHLIX
-> BLENDERBOT:  WHAT HAPPENED? DID YOU KNOW IT WAS YOUR FRIENDS?



-> MIGUEL: MY FRIENDS WHAT HAPPENED WITH MY FRIENDS
-> BLENDERBOT:  I'm sorry to hear that.  What happened?  I hope it wasn't too bad.



-> MIGUEL: THING BUT HAPPENED A DO YOU WANT TO HAVE VENER LATER AFTER WORK
-> BLENDERBOT:  I don't think I want to go back to school. I feel like I'm wasting my time.



-> MIGUEL: WHEN ARE YOU COMING BACK HOME
-> BLENDERBOT:  I am going to the beach!  I am so excited.  I have never been on a cruise before.
